In [ ]:
%load_ext autoreload
from mps.state import *

# TIME EVOLUTION

In [ ]:
# file: mps/evolution.py

import numpy as np
import scipy.linalg
from numbers import Number
import mps.state
import scipy.sparse as sp
from mps.state import _truncate_vector, DEFAULT_TOLERANCE

Some of the commonly used operators:

In [ ]:
# file: mps/evolution.py

σz = np.diag([1.0,-1.0])
i2 = np.identity(2)
σx = np.array([[0, 1], [1, 0]])
σy = -1j * σz @ σx

def creation_op(d):
    # Returns d dimensional cration operator
    return np.diag(np.sqrt(np.arange(1,d)),-1).astype(complex)

def annihilation_op(d):
    # Returns d dimensional cration operator
    return np.diag(np.sqrt(np.arange(1,d)),1).astype(complex)



## Nearest Neighbor Hamiltonians

Below is the interface we use for nearest neighbor interaction Hamiltonians. It is initially empty.

In [ ]:
# file: mps/evolution.py

class NNHamiltonian(object):
    
    def __init__(self, size):
        #
        # Create a nearest-neighbor interaction Hamiltonian
        # of a given size, initially empty.
        #
        self.size = size
        
    def dimension(self, ndx):
        #
        # Return the dimension of the local Hilbert space
        #
        return 0
    
    def interaction_term(self, ndx, t=0.0):
        #
        # Return the interaction between sites (ndx,ndx+1)
        #
        return 0

Generally, we can have different local terms and different interactions on every site
$$
H = \sum_i \left[O_i + \sum_n L^{(n)}_i \otimes R^{(n)}_{i+1}\right]_\text{site i} 
=  \sum_{i=0}^{N-2} h_{i,i+1}. 
$$
where
$$
h_{i,i+1} = \sum_n L^{(n)}_i \otimes R^{(n)}_{i+1} +
\begin{cases}
O_i + \frac{1}{2} O_{i+1}, \text{ if  } i = 0 \\
\frac{1}{2} O_i + O_{i+1}, \text{ if  } i = N-2 \\
\frac{1}{2} O_i + \frac{1}{2} O_{i+1}, \text{ else  } 
\end{cases}
$$
and $N>2$.

The function below computes the interaction terms $h_{i,i+1}$:

In [ ]:
# file: mps/evolution.py


class ConstantNNHamiltonian(NNHamiltonian):

    def __init__(self, size, dimension):
        #
        # Create a nearest-neighbor interaction Hamiltonian with fixed
        # local terms and interactions.
        #
        #  - local_term: operators acting on each site (can be different for each site)
        #  - int_left, int_right: list of L and R operators (can be different for each site)
        #
        self.size = size
        self.int_left = [[]] * (size-1)
        self.int_right = [[]] * (size-1)
        if isinstance(dimension, Number):
            dimension = [dimension] * size
        self.dimension_ = dimension

    def set_local_term(self, ndx, operator):
        #
        # Set the local term acting on the given site
        #
        if ndx == 0:
            self.add_interaction_term(ndx, operator, np.eye(self.dimension(1)))
        elif ndx == self.size-2:
            self.add_interaction_term(ndx, np.eye(self.dimension(ndx)), operator)
        else:
            self.add_interaction_term(ndx, np.eye(self.dimension(ndx)), 0.5*operator)
            self.add_interaction_term(ndx, 0.5*operator, np.eye(self.dimension(ndx+1)))

    def add_interaction_term(self, ndx, L, R):
        #
        # Add an interaction term $L \otimes R$ acting on sites 'ndx' and 'ndx+1'
        #
        # Add to int_left, int_right
        #
        # Update the self.interactions[ndx] term
        self.int_left[ndx].append(L)
        self.int_right[ndx].append(R)

    def dimension(self, ndx):
        return self.dimension_[ndx]

    def interaction_term(self, ndx, t=0.0):
        #for (L, R) in zip(self.int_left[ndx], self.int_right[ndx]):
            
        return sum([np.kron(L, R) for (L, R) in zip(self.int_left[ndx], self.int_right[ndx])])
            

A particular case would be a translationally invariant, constant Hamiltonian
$$H = \sum_i \left[O + \sum_n L^{(n)} \otimes R^{(n)}\right]_\text{site i}$$
which has the same local term $O$ on all sites, and the same interaction given by the product of $L^{(n)}$ left and $R^{(n)}$ right operators.

In [ ]:
# file: mps/evolution.py

def make_ti_Hamiltonian(size, intL, intR, local_term=None):
    """Construct a translationally invariant, constant Hamiltonian with open
    boundaries and fixed interactions.
    
    Arguments:
    size        -- Number of sites in the model
    int_left    -- list of L (applied to site ndx) operators
    int_right   -- list of R (applied to site ndx + 1) operators
    local_term  -- operator acting on every site (optional)
    
    Returns:
    H           -- ConstantNNHamiltonian
    """
    if local_term is not None:
        dimension = len(local_term)
    else:
        dimension = len(intL[0])
    
    H = ConstantNNHamiltonian(size, dimension)
    H.local_term = local_term
    H.intL = intL
    H.intR = intR
    for ndx in range(size-1):
        for L,R in zip(H.intL, H.intR):
            H.add_interaction_term(ndx, L, R)
        if local_term is not None:
            H.set_local_term(ndx, local_term)
    return H

## Suzuki-Trotter Decomposition

In Suzuki-Trotter decomposition, the Hamiltonians of the nearest neighbor couplings can be decomposed into two non-commuting parts, $H_{\text{odd}} $ and $ H_{\text{even}} $, so that all additive 2-site operators in each part commute with each other.

Let us consider a simple example of tight binding model with on-site potential and decompose the Hamiltonian into 2-site terms, so that $H=\sum_i h_{i,i+1}$. 
\begin{equation}
h_{i,i+1} = \left(\frac{\omega}{2}  a_i^\dagger a_i \right) + \left(\frac{\omega}{2}  a_{i+1}^\dagger a_{i+1} \right) - \left( t a_{i}^\dagger a_{i+1} + \text{h.c.} \right).
\end{equation}
Since $[h_{i,i+1},h_{i+2,i+3}] = 0$, we can group these terms for even and odd $i$, so that $H = H_{\text{odd}} + H_{\text{even}} $. 

Note that the local term $ a_i^\dagger a_i$ appears only in one the groups for $i=1$ and $i=N$. Therefore we need to add two on-site terms $h_1 = \left(\frac{\omega}{2}  a_1^\dagger a_1 \right) $ and $h_N = \left(\frac{\omega}{2}  a_N^\dagger a_N \right) $, to the corresponding two-site terms. So that $h_{1,2} \rightarrow h_{1,2} + h_1$, and $h_{N-1,N} \rightarrow h_{N-1,N} + h_N$.

And for the first order Suzuki-Trotter decomposition, the evolution operator becomes
\begin{equation}
e^{-i \hat{H} \Delta t} = e^{-i \hat{H}_{\text{odd}} \Delta t}  e^{-i \hat{H}_{\text{even}} \Delta t} + O(\Delta t^2).
\end{equation}

`pairwise_unitaries` creates a list of Trotter unitarities corresponding to two-site operators, $U_{i,i+1} = e^{-i h_{i,i+1} \Delta t}$. The Trotter unitarities associated with $\hat{H}_{\text{odd}}$ and $\hat{H}_{\text{even}}$ are applied separately in consecutive sweeps depending on evenodd value passed to TEBD_sweep class:
$$ U = [U_{1,2}, U_{2,3}, U_{3,4}, \dots ]. $$


In [ ]:
# file: mps/evolution.py


def pairwise_unitaries(H, δt):
    return [scipy.linalg.expm((-1j * δt) * H.interaction_term(k)).
                              reshape(H.dimension(k), H.dimension(k+1),
                                      H.dimension(k), H.dimension(k+1))
            for k in range(H.size-1)]

We apply each $U_{i,i+1} = e^{-i h_{i,i+1} \Delta t}$ to two neighbouring tensors, $A_i$ and $A_{i+1}$ simultaneously, as shown below.

<img src="fig_pdf/apply_mpo_to2site.svg" style="max-width: 90%; width: 35em">

The resulting tensor $B$ is a two-site tensor. We split this tensor into two one-site tensors using left_orth_2site and right_orth_2site functions, which are defined in [this notebook](File%201c%20-%20Canonical%20form.ipynb). 

In [ ]:
# file: mps/evolution.py

def apply_2siteTrotter(U, ψ, start):
    return np.einsum('ijk,klm,prjl -> iprm', ψ[start], ψ[start+1], U)


In [ ]:
# file: mps/evolution.py


def TEBD_sweep(U, ψ, tol=DEFAULT_TOLERANCE):
    #
    # Apply a TEBD sweep by evolving with the pairwise Trotter Hamiltonian
    # starting from left/rightmost site and moving on the 'direction' (>0 right,
    # <0 left) by pairs of sites.
    #
    # - H: NNHamiltonian
    # - ψ: Initial state in CanonicalMPS form (modified destructively)
    # - δt: Time step
    # - evenodd: 0, 1 depending on Trotter step
    # - direction: where to move
    #
    if ψ.center <= 2:
        dr = +1
        if ψ.center == 0:
            evenodd = 0
        else:
            evenodd = 1
    else:
        dr = -1
        evenodd = ψ.size % 2
        if ψ.center < ψ.size - 1:
            evenodd = 1-evenodd

    def update_two_site(start, nextsite, dr):
        # Apply combined local and interaction exponential and move
        if start == 0:
            dr = +1
        elif start == (ψ.size-2):
            dr = -1
        AA = apply_2siteTrotter(U[start], ψ, start)
        ψ.update_canonical_2site(AA, start, nextsite, dr, tolerance=tol)
        print('updating sites ({}, {}), center={}, direction={}'.format(
            start, nextsite, ψ.center, dr))

    #
    # Loop over ψ, updating pairs of sites acting with the unitary operator
    # made of the interaction and 0.5 times the local terms
    #
    if dr < 0:
        if ψ.size % 2 == evenodd:
            start = ψ.size - 1
        else:
            start = ψ.size - 2
        #print('TEBD sweep with direction {} and start {}'.format(dr, start))
        for j in range(start, 0, -2):
            update_two_site(j-1, j, -1)
    else:
        start = 0 + evenodd
        #print('TEBD sweep with direction {} and start {}'.format(dr, start))
        for j in range(start, ψ.size-1, +2):
            update_two_site(j, j+1, +1)

    return ψ

In [ ]:
# file: mps/evolution.py


class TEBD_evolution(object):
    def __init__(self, H, dt, timesteps=1, order=1, tol=DEFAULT_TOLERANCE):
        self.H = H
        self.dt = float(dt)
        self.timesteps = timesteps
        self.order = order
        self.tolerance = tol
        self.Udt = pairwise_unitaries(H, dt)
        if order == 2:
            self.Udt2 = pairwise_unitaries(H, dt/2)

    def evolve(self, ψ):
        if not isinstance(ψ, mps.state.CanonicalMPS):
            evenodd = 0
            dr = 1
            ψ = mps.state.CanonicalMPS(ψ, center=0)
        for i in range(self.timesteps):
            if self.order == 1:
                newψ = TEBD_sweep(self.Udt, ψ, tol=self.tolerance)
                newψ = TEBD_sweep(self.Udt, newψ, tol=self.tolerance)
            else:
                newψ = TEBD_sweep(self.Udt2, ψ, tol=self.tolerance)
                newψ = TEBD_sweep(self.Udt, newψ, tol=self.tolerance)
                newψ = TEBD_sweep(self.Udt2, newψ, tol=self.tolerance)
        return newψ

## Error in Suzuki-Trotter decomposition

In the first order Suzuki-Trotter decomposition, evolution operator becomes
\begin{equation}
e^{-i \hat{H} \Delta t} = e^{-i \hat{H}_{\text{odd}} \Delta t}  e^{-i \hat{H}_{\text{even}} \Delta t} + O(\Delta t^2).
\end{equation}
Note that after $T/\Delta t$ time steps, the accumulated error is in the order of $\Delta t$.
Higher order Suzuki-Trotter decompositions can be used to reduce error.




# Tests

In [ ]:
# file: mps/test/test_TEBD.py
from mps.evolution import *


In [ ]:
# file: mps/test/test_TEBD.py
import unittest
import mps.state
import mps.tools
from mps.test.tools import *
import scipy.sparse as sp

def random_wavefunction(n):
    ψ = np.random.rand(n) - 0.5
    return ψ / np.linalg.norm(ψ)

class TestTEBD_sweep(unittest.TestCase):
        
    def test_TEBD_evolution_first_order(self):
        #
        #
        #
        N = 19
        t = 0.1
        ω = 0.5
        dt = 1e-7
        Nt = int(10000)
        ψwave = random_wavefunction(N)
        ψmps = CanonicalMPS(mps.state.wavepacket(ψwave))
        # We use the tight-binding Hamiltonian
        H=make_ti_Hamiltonian(N, [t * annihilation_op(2) , t * creation_op(2)], 
                            [creation_op(2), annihilation_op(2)], 
                              local_term=ω*creation_op(2)@ annihilation_op(2))
        #for i in range(Nt):
        #    
        #    ψmps = TEBD_sweep(H, ψmps, dt, 1, 0, tol=DEFAULT_TOLERANCE)
        #    ψmps = TEBD_sweep(H, ψmps, dt, -1, 1, tol=DEFAULT_TOLERANCE)
        ψmps = TEBD_evolution(H, dt, timesteps=Nt, order=1, tol=DEFAULT_TOLERANCE).evolve(ψmps)
        Hmat = sp.diags([[t]*(N), ω, [t]*(N)],
                  offsets=[-1,0,+1],
                  shape=(N,N),
                  dtype=np.complex128)
        
        ψwave_final = sp.linalg.expm_multiply(-1j * dt*Nt * Hmat, ψwave)
        
        self.assertTrue(similar(abs(mps.state.wavepacket(ψwave_final).tovector()), 
                                abs(ψmps.tovector())))
                

    def test_TEBD_evolution_second_order(self):
        #
        #
        #
        N = 21
        t = 0.1
        ω = 0.5
        dt = 1e-7
        Nt = int(100)
        ψwave = random_wavefunction(N)
        ψmps = CanonicalMPS(mps.state.wavepacket(ψwave))
        # We use the tight-binding Hamiltonian
        H=make_ti_Hamiltonian(N, [t * annihilation_op(2) , t * creation_op(2)], 
                            [creation_op(2), annihilation_op(2)], 
                              local_term=ω*creation_op(2)@ annihilation_op(2))
        #for i in range(Nt):
        #    
        #    ψmps = TEBD_sweep(H, ψmps, dt, 1, 0, tol=DEFAULT_TOLERANCE)
        #    ψmps = TEBD_sweep(H, ψmps, dt, -1, 1, tol=DEFAULT_TOLERANCE)
        ψmps = TEBD_evolution(H, dt, timesteps=Nt, order=2, tol=DEFAULT_TOLERANCE).evolve(ψmps)
        Hmat = sp.diags([[t]*(N), ω, [t]*(N)],
                  offsets=[-1,0,+1],
                  shape=(N,N),
                  dtype=np.complex128)
        
        ψwave_final = sp.linalg.expm_multiply(-1j * dt*Nt * Hmat, ψwave)
        
        self.assertTrue(similar(abs(mps.state.wavepacket(ψwave_final).tovector()), 
                                abs(ψmps.tovector())))
                


    

In [ ]:
%autoreload
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestTEBD_sweep'])
unittest.TextTestRunner(verbosity=2).run(suite1);